In [11]:
from nocodb_base import fetch_all_rows_from_nocodb, update_row
from file_utils import write_json
import re


In [12]:
# Configuration
xc_token = ""
table_id = "mlb3px3mdzkxpuh"
view_id = "vwae4bvxtzxoegv8"
noco_base_url = ""
headers = {
    'accept': 'application/json',
    'xc-token': xc_token,
}
index_file = "index_check_20250801/index_sorted.cdxj"

In [13]:
# Configuration
xc_token = "G-sFojI-efGYr3WlfBAJLQt3vSnrIK0lkuE7FlB_"
# all cloudfront urls
#table_id = "mxp572bg586eppw"
#view_id = "vwy8ovu97qes5d9h"
# all pages
table_id = "mocva6wq3lfizyp"
view_id = "vwdjsszj1rpxh7tt"
noco_base_url = "http://localhost:8080"
headers = {
    'accept': 'application/json',
    'xc-token': xc_token,
}
index_file = "index_check_20250802/index_sorted.cdxj"


In [14]:
rows_data = fetch_all_rows_from_nocodb(noco_base_url, headers, table_id, view_id)
#print(rows_data)

test1
Fetching data from offset 0...
test3
Fetched 1000 posts from offset 0
test1
Fetching data from offset 1000...
test3
Fetched 921 posts from offset 1000
Total posts fetched: 1921


In [15]:
from urllib.parse import urlparse, urlunparse, quote

def normalize_url(url):
    url = url.strip()
    parsed = urlparse(url)

    scheme = parsed.scheme.lower()
    netloc = parsed.netloc.lower()

    # Lowercase path, strip trailing slash
    path = parsed.path.rstrip('/').lower()

    # Keep query string if present (as-is)
    query = parsed.query

    # Rebuild the normalized URL
    normalized = urlunparse((scheme, netloc, path, '', query, ''))

    # Encode safely (optional but helpful if comparing to encoded index)
    return quote(normalized, safe=':/?&=%')

In [16]:
# Load index file and extract URLs
urls_index = {}
url_pattern = re.compile(r'"url":\s*"([^"]+)"')

with open(index_file, 'rt', encoding='utf-8') as f:
    for line in f:
        match = url_pattern.search(line)
        if match:
            url = normalize_url(match.group(1))
            urls_index[url] = True

In [17]:
# Process rows and add index status
urls = []
index_check_field = "index_check"
for row in rows_data:
    record_id = row.get("Id")  # Adjust if your ID column has a different name
    url = row.get("URL")  # Adjust if your URL column has a different name
    normalized_url = normalize_url(url)
    
    if normalized_url in urls_index:
        index_status = "in index"
        #row["index_status"] = "in index"
    else:
        index_status = "not in index"
        #print(url)
        urls.append({"id": record_id, "url": url}) 
        #row["index_status"] = "not in index"
    
    update_row(noco_base_url, headers, table_id, record_id, index_status,index_check_field)

# Save updated rows
#output_file = "cloudfront_not_in_index.json"
output_file = "pages_not_in_index.json"

write_json(output_file, urls)
#print(f"Updated rows saved to {output_file}")

JSON data written to pages_not_in_index.json
